In [1]:
import scvelo as scv
import dynamo as dyn
import numpy as np
from anndata import AnnData
# import loompy
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from scipy.cluster.hierarchy import fcluster,leaders
from sklearn.decomposition import PCA
from scipy.linalg import inv
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.mixture import GaussianMixture
from RKHS import SparseVFC
from RKHS import Jacobian_rkhs_gaussian
from scipy.stats import multivariate_normal
from scipy.sparse import csr_matrix
import pandas as pd

In [2]:
import argparse
import random

import scipy.sparse as sp
import scipy.sparse.csgraph
import sklearn.linear_model as sklm
import sklearn.metrics as skm
import sklearn.model_selection as skms
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from ignite.engine import Engine, Events
from ignite.handlers import ModelCheckpoint
from torch.utils.data import DataLoader, IterableDataset, get_worker_info

CHECKPOINT_PREFIX = "g2g"

from g2g_model_Fisher import *
from utils import *
# from minepy import MINE
from sklearn.preprocessing import MinMaxScaler

In [3]:
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

1.10.0
11.3


True

In [4]:
data_path='data/'
result_path='results/'
adata0=scv.read('DG_bin.h5ad', cache=True)

In [ ]:
gene_arr=adata0.var.index.values
X_pca=adata0.obsm['X_pca']
X_umap=adata0.obsm['X_umap']
cell_vpt=adata0.obs['velocity_pseudotime'].values#adata0.obs['latent_time'].values#
Xs=adata0.layers['Ms']#adata.X.A#
X_pca.shape

In [ ]:
#-------data preprocessing
k_nei=10
adata=adata0.copy()
scv.pp.neighbors(adata, n_neighbors=k_nei)
scv.pp.pca(adata,n_comps=50)
scv.pp.moments(adata, n_pcs=50, n_neighbors=k_nei)

In [ ]:
cell_nei = np.load('results/cell_nei.npy')
nei_w = np.load('results/nei_w.npy')

def smooth_func(X_val,cell_nei=cell_nei,nei_w=nei_w):
    X_s=X_val.copy()
    for ci in range(len(X_val)):
        X_s[ci]=np.dot(X_val[cell_nei[ci,:]],nei_w[ci,:])
    return(X_s)

In [ ]:
encoder = torch.load('results/encoder.pt')
# encoder = torch.load('results/encoder L=4,K=1.pt')

In [114]:
# X=Xs
X=Xs/np.mean(np.abs(Xs),axis=0)
# X=adata.X.A
mu, sigma = encoder(torch.tensor(X))
mu_learned = mu.detach().numpy()
sigma_learned = sigma.detach().numpy()
latent_z = np.hstack((mu_learned,sigma_learned))

L = np.load('results/latent_dim.npy')
Fisher_g=np.zeros((X.shape[0],L*2,L*2))
for i in range(X.shape[0]):
    for j in range(L):
        Fisher_g[i,j,j]=1/sigma_learned[i,j]**2
        Fisher_g[i,L+j,L+j]=2/sigma_learned[i,j]**2

In [ ]:
crc = np.load('results/crc.npy')
crc_eu = np.load('results/crc_eu.npy')
crc_smooth = np.load('results/crc_smooth.npy')

In [112]:
#------using velocity_pca to calculate dmu/dt and dsigma/dt
pca_dim=20
velo=adata.obsm['velocity_pca'][:,:pca_dim]#np.array(adata.layers['velocity'])
#velo=velo/np.std(np.abs(velo),axis=0)#
# plt.hist(velo.flatten(),bins=200)
# plt.show()
X=X_pca[:,:pca_dim]#scaler.fit_transform(Xs)#[:,adata.var['velocity_genes'].values]
# for k in range(pca_dim):
#     plt.scatter(X_pca[:,0],X_pca[:,1],s=5,c=velo[:,k],cmap=plt.cm.jet)
#     plt.xlabel('PC1',fontsize=14)
#     plt.ylabel('PC2',fontsize=14)

#     clb=plt.colorbar()

#     plt.show()

In [ ]:
# #------------use RKHS to obtain the dmu/dt and dsigma/dt

# # vec_F=SparseVFC(X,latent_z,Grid=X)
# # pLz_pX=Jacobian_rkhs_gaussian(X, vec_F)
# # pLz_pX=np.transpose(np.array(pLz_pX),[2,0,1])
# # pLz_pX.shape

# vec_Mu=SparseVFC(X,X_mu,Grid=X)
# pMu_pX=Jacobian_rkhs_gaussian(X, vec_Mu)
# pMu_pX=np.transpose(np.array(pMu_pX),[2,0,1])
# vec_Sgm=SparseVFC(X,X_sigma,Grid=X)
# pSgm_pX=Jacobian_rkhs_gaussian(X, vec_Sgm)
# pSgm_pX=np.transpose(np.array(pSgm_pX),[2,0,1])

# mu_velo = np.array([pMu_pX[i]@velo[i] for i in range(X.shape[0])])
# sgm_velo = np.array([pSgm_pX[i]@velo[i] for i in range(X.shape[0])])
# zv2 = np.zeros(X.shape[0])
# for i in range(X.shape[0]):
#     for j in range(L):
#         zv2[i] += Fisher_g[i,2*j,2*j]*mu_velo[i,j]**2 + Fisher_g[i,2*j+1,2*j+1]*sgm_velo[i,j]**2
# zv1 = np.sqrt(zv2)

# plt.scatter(X_pca[:,0],X_pca[:,1],s=5,c=np.linalg.norm(np.hstack((mu_velo,sgm_velo)),axis=1),cmap=plt.cm.jet)
# plt.xlabel('PC1',fontsize=14)
# plt.ylabel('PC2',fontsize=14)

# clb=plt.colorbar()

# plt.show()
# for k in range(L):
#     plt.scatter(X_pca[:,0],X_pca[:,1],s=5,c=mu_velo[:,k],cmap=plt.cm.jet)
#     plt.xlabel('PC1',fontsize=14)
#     plt.ylabel('PC2',fontsize=14)
#     clb=plt.colorbar()
#     plt.show()

#     plt.scatter(X_pca[:,0],X_pca[:,1],s=5,c=sgm_velo[:,k],cmap=plt.cm.jet)
#     plt.xlabel('PC1',fontsize=14)
#     plt.ylabel('PC2',fontsize=14)
#     clb=plt.colorbar()
#     plt.show()

In [ ]:
#------use a simple neural network to study dmu/dt and dsigma/dt
from torch.optim import SGD

model = nn.Sequential(
    nn.Linear(X.shape[1], 128),
    nn.ReLU(),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.Linear(64,2*L),
)

# Define your loss function and optimizer
loss_fn = nn.MSELoss()
optimizer = SGD(model.parameters(), lr=0.01, weight_decay=0.001)#, momentum=0.2

x_in=torch.tensor(X)
x_out=torch.tensor(latent_z)
# Train the model
for epoch in range(500):  # number of epochs
    # Forward pass
    output = model(x_in)
    loss = loss_fn(output,x_out) 
    # Backward pass and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

pZ_pX = np.zeros([X.shape[0], L*2, X.shape[1]])

# Compute the gradients
for i in range(X.shape[0]):
    x0=torch.tensor(X[i,:],requires_grad=True)
    z=model(x0)
    for j in range(2*L):
        x0.grad = None       
        z[j].backward(retain_graph=True)
        pZ_pX[i,j,:] = x0.grad.detach()


Z_velo = np.array([pZ_pX[i]@velo[i] for i in range(X.shape[0])])

zv2 = np.zeros(X.shape[0])
for i in range(X.shape[0]):
    for j in range(L):
        # zv2[i] += Fisher_g[i,2*j,2*j]*Z_velo[i,2*j]**2 + Fisher_g[i,2*j+1,2*j+1]*Z_velo[i,2*j+1]**2
        zv2[i] += Fisher_g[i,j,j]*Z_velo[i,j]**2 + Fisher_g[i,L+j,L+j]*Z_velo[i,L+j]**2
zv1 = np.sqrt(zv2)
zv1_smooth = smooth_func(zv1)

# for k in range(2*L):
#     plt.scatter(X_pca[:,0],X_pca[:,1],s=5,c=Z_velo[:,k],cmap=plt.cm.jet)
#     plt.xlabel('PC1',fontsize=14)
#     plt.ylabel('PC2',fontsize=14)

#     clb=plt.colorbar()

#     plt.show()
    
# plt.scatter(X_pca[:,0],X_pca[:,1],s=5,c=np.linalg.norm(Z_velo,axis=1),cmap=plt.cm.jet)
# plt.xlabel('PC1',fontsize=14)
# plt.ylabel('PC2',fontsize=14)

# clb=plt.colorbar()

# plt.show()

In [ ]:
plt.scatter(adata.obsm['X_pca'][:,0],adata.obsm['X_pca'][:,1],s=5,c=zv1,cmap=plt.cm.jet)
plt.xlabel('PC1',fontsize=14)
plt.ylabel('PC2',fontsize=14)
clb=plt.colorbar()
clb.ax.set_ylabel('Information velocity',fontsize=14)
plt.savefig(result_path+'iv_pca.png')
plt.show()

plt.scatter(adata.obsm['X_pca'][:,0],adata.obsm['X_pca'][:,1],s=5,c=zv1_smooth,cmap=plt.cm.jet)
plt.xlabel('PC1',fontsize=14)
plt.ylabel('PC2',fontsize=14)
clb=plt.colorbar()
clb.ax.set_ylabel('Information velocity',fontsize=14)
plt.savefig(result_path+'iv_smooth_pca.png')
plt.show()

# plt.scatter(X_sigma[:,0],X_sigma[:,1], c=zv1, s=2, alpha=0.8,cmap=plt.cm.jet)
# plt.colorbar()
# plt.show()

In [ ]:
plt.figure(dpi=600)
ln1 = plt.scatter(cell_vpt,crc_smooth,c='blue',label='curvature')
plt.xlabel('velovity psuedotime',fontsize=14,weight='bold')
plt.ylabel('curvature',fontsize=14,weight='bold')
plt.twinx()
ln2 = plt.scatter(cell_vpt,zv1_smooth,c='orange',label='info velo')
plt.ylabel('info velo',fontsize=14,weight='bold')
plt.legend(handles=[ln1,ln2])
plt.savefig(result_path+'cviv_pca.png')
plt.show()

In [ ]:
mask,x_ls,y_ls= kde_lowess(cell_vpt,crc_smooth)
plt.scatter(cell_vpt[mask], crc_smooth[mask],s=10,c=cell_vpt[mask],cmap=plt.cm.jet)
plt.plot(x_ls, y_ls, linewidth=4,color='red')
plt.xlabel('pseudotime',fontsize=14)
plt.ylabel('curvature',fontsize=14)
# sns.kdeplot(cell_vpt,cRc_smooth,bw_adjust=0.7)
clb=plt.colorbar()
clb.ax.set_ylabel('pseudotime',fontsize=14)
# plt.savefig(result_path+'cv_kde.png')
plt.show()

mask,x_ls,y_ls= kde_lowess(cell_vpt,zv1_smooth)
plt.scatter(cell_vpt[mask], zv1_smooth[mask],s=10,c=cell_vpt[mask],cmap=plt.cm.jet)
plt.plot(x_ls, y_ls, linewidth=4,color='red')
plt.xlabel('pseudotime',fontsize=14)
plt.ylabel('information velocity',fontsize=14)
# sns.kdeplot(cell_vpt,cRc_smooth,bw_adjust=0.7)
clb=plt.colorbar()
clb.ax.set_ylabel('pseudotime',fontsize=14)
plt.savefig(result_path+'iv_kde_pca.png')
plt.show()

In [ ]:
plt.figure(dpi=600)
mask,x_ls,y_ls= kde_lowess(cell_vpt,crc_smooth)
plt.scatter(cell_vpt[mask], crc_smooth[mask],s=10,c='blue',label='curvature')
plt.plot(x_ls, y_ls, linewidth=4,color='red')
plt.xlabel('velovity psuedotime',fontsize=14,weight='bold')
plt.ylabel('curvature',fontsize=14,weight='bold')
plt.twinx()
mask,x_ls,y_ls= kde_lowess(cell_vpt,zv1_smooth)
plt.scatter(cell_vpt[mask], zv1_smooth[mask],s=10,c='orange',label='info velo')
plt.plot(x_ls, y_ls, linewidth=4,color='red')
plt.legend(handles=[ln1,ln2])
plt.savefig(result_path+'cviv_kde_pca.png')
plt.show()

In [128]:
g_pc = np.zeros([X.shape[0],pca_dim,pca_dim])
for i in range(X.shape[0]):
    g_pc[i] = pZ_pX[i].T @ Fisher_g[i] @ pZ_pX[i]

In [ ]:
plt.imshow(np.mean(g_pc,axis=0),cmap=plt.cm.jet)
plt.colorbar()
plt.show()

# for j in range(pca_dim):
#     plt.scatter(cell_vpt,g_pc[:,j,j],c=cell_vpt,cmap=plt.cm.jet)
# #         plt.yscale('log')
#     plt.show()

# for i in range(pca_dim):
#     plt.scatter(adata.obsm['X_pca'][:,0],adata.obsm['X_pca'][:,1],s=5,c=g_pc[:,i,i],cmap=plt.cm.jet)
#     plt.xlabel('PC1',fontsize=14)
#     plt.ylabel('PC2',fontsize=14)
    
#     clb=plt.colorbar()

#     clb.ax.set_ylabel('Fisher matrix',fontsize=14)
# #     plt.savefig(result_path+'g_'+str(i)+'.png')
#     plt.show()

In [ ]:
cs=['r','b','k','g','y','purple','cyan']
rinds=np.random.choice(g_pc.shape[0],size=800)
for xi in rinds:#range(g_pc.shape[0]):
    fim_eigen=np.sort(np.linalg.eig(g_pc[xi,:,:])[0])[-6:]
    fim_eigen=np.real(fim_eigen/np.amax(fim_eigen))
#     print(fim_eigen)
    for j in range(6):
        plt.plot(cell_vpt[xi],fim_eigen[j],'.',color=cs[j])
        plt.yscale('log')
plt.show()

# for xi in rinds:#range(g_pc.shape[0]):
#     fim_eigen=np.sort(g_pc[xi,:,:].flatten())[-2:]
#     fim_eigen=fim_eigen/np.amax(fim_eigen)
    
#     for j in range(fim_eigen.shape[0]):
#         plt.plot(cell_vpt[xi],fim_eigen[j],'.')
#         plt.yscale('log')
# plt.show()
    

In [130]:
# vec_F=SparseVFC(latent_z,X_pca,Grid=latent_z)
# #partial derivtive of Mu of each Gmm component with repect to each eigen gene
# pPc_pZ=Jacobian_rkhs_gaussian(latent_z, vec_F)
# pPc_pZ=np.transpose(np.array(pPc_pZ),[2,0,1])

model0 = nn.Sequential(
    nn.Linear(2*L, 128),
    nn.ReLU(),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.Linear(64,X.shape[1]),
)

# Define your loss function and optimizer
# loss_fn = nn.MSELoss()
optimizer0 = SGD(model0.parameters(), lr=0.01, weight_decay=0.001)#, momentum=0.2

x_in=torch.tensor(latent_z)
x_out=torch.tensor(X)
# Train the model
for epoch in range(500):  # number of epochs
    # Forward pass
    output = model0(x_in)
    loss = loss_fn(output,x_out) 
    # Backward pass and optimize
    optimizer0.zero_grad()
    loss.backward()
    optimizer0.step()

pX_pZ = np.zeros([X.shape[0], X.shape[1], L*2])

zv20 = np.zeros(X.shape[0])
for i in range(X.shape[0]):
    zvi = pX_pZ[i]@Z_velo[i]
    zv20[i] = zvi@g_pc[i]@zvi
zv10 = np.sqrt(zv20)
zv10_smooth = smooth_func(zv10,cell_nei,nei_w)

In [ ]:
plt.scatter(adata.obsm['X_pca'][:,0],adata.obsm['X_pca'][:,1],s=5,c=zv10,cmap=plt.cm.jet)
plt.xlabel('PC1',fontsize=14)
plt.ylabel('PC2',fontsize=14)
clb=plt.colorbar()
clb.ax.set_ylabel('Information velocity',fontsize=14)
plt.savefig(result_path+'iv_pca.png')
plt.show()

plt.scatter(adata.obsm['X_pca'][:,0],adata.obsm['X_pca'][:,1],s=5,c=zv10_smooth,cmap=plt.cm.jet)
plt.xlabel('PC1',fontsize=14)
plt.ylabel('PC2',fontsize=14)
clb=plt.colorbar()
clb.ax.set_ylabel('Information velocity',fontsize=14)
plt.savefig(result_path+'iv_smooth_pca.png')
plt.show()

In [ ]:
plt.figure(dpi=600)
ln1 = plt.scatter(cell_vpt,crc_smooth,c='blue',label='curvature')
plt.xlabel('velovity psuedotime',fontsize=14,weight='bold')
plt.ylabel('curvature',fontsize=14,weight='bold')
plt.twinx()
ln2 = plt.scatter(cell_vpt,zv10_smooth,c='orange',label='info velo 0')
plt.ylabel('info velo',fontsize=14,weight='bold')
plt.legend(handles=[ln1,ln2])
plt.show()

plt.figure(dpi=600)
ln1 = plt.scatter(cell_vpt,zv1_smooth,c='blue',label='info velo')
plt.xlabel('velovity psuedotime',fontsize=14,weight='bold')
plt.ylabel('curvature',fontsize=14,weight='bold')
plt.twinx()
ln2 = plt.scatter(cell_vpt,zv10_smooth,c='orange',label='info velo 0')
plt.ylabel('info velo',fontsize=14,weight='bold')
plt.legend(handles=[ln1,ln2])
plt.show()

In [ ]:
g_pc_ev = np.array([np.linalg.eigvals(g_pc[i]) for i in range(X.shape[0])])

for i in range(pca_dim):
    plt.scatter(adata.obsm['X_pca'][:,0],adata.obsm['X_pca'][:,1],s=5,c=g_pc_ev[:,i],cmap=plt.cm.jet)
    plt.xlabel('PC1',fontsize=14)
    plt.ylabel('PC2',fontsize=14)
    
    clb=plt.colorbar()

    clb.ax.set_ylabel('Fisher matrix',fontsize=14)
#     plt.savefig(result_path+'g_'+str(i)+'.png')
    plt.show()